In [ ]:
!which python; python -V;
from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.ephem import Ephem

from perylune.orbit_tools import *
from perylune.mpc import *

In [ ]:
# First step is to load the data. parse_txt() will load the orbital elements from text file in MPC format.
# This will be returned as an array. Each entry in that array has a list of 8 parameters or so.
elements = parse_txt("../data/mpc/mpcorb_extended.dat", limit=0, skip="------------------")

print("Number of orbital elements parsed: %d" % len(elements))

In [ ]:
# Now process the data. Find min and max a and e, also put the a values in 
max_e = 0
min_e = 9999999
max_a = 0
min_a = 9999999

a_tbl = []
e_tbl = []
cnt = 0
for e in elements:
    a = e[8].value
    ecc = e[6].value
    if a < 6:
        cnt = cnt + 1
        a_tbl.append(a)
        e_tbl.append(e)
    if max_a < a:
        max_a = a
    if min_a > a:
        min_a = a
    if max_e < ecc:
        max_e = ecc
    if min_e > ecc:
        min_e = ecc

print ("Processed %d asteroids, a=%f..%f [AU], eccentricity = %f..%f" % (cnt, min_a, max_a, min_e, max_e))

In [ ]:
# First, let's take a look at the major semiaxis. As seen here, the majority of the asteroids are in the main asteroid belt.
# The histogram is truncated at 6 AU. There are some TNO (Trans-Neptunian Objects) and KBO (Kuiper Belt Objects) that 
# are very far away
import plotly.graph_objects as go

fig = go.Figure(data=[go.Histogram(x=np.array(a_tbl))])
fig.update_layout(title_text='Distribution of %d asteroids\' semi major-axis' % len(a_tbl), xaxis_title_text='Major semiaxis (AU)', yaxis_title_text='Count')
fig.show()

In [ ]:
print(len(e_tbl))
print(e_tbl[0])
#fig2 = go.Figure(data=[go.Histogram(x=np.array(e_tbl))])
#fig2.update_layout(title_text='Solar System Asteroids (%d)' % len(e_tbl), xaxis_title_text='Eccentricity []', yaxis_title_text='Count')
#fig2.show()

In [ ]:
t2 = time.time()
dv_tbl2 = sorted(dv_tbl, key=lambda e: e[1].value)
t3 = time.time()

print("dv_tbl is now sorted, sorting took %s s." % (t3-t2))

In [ ]:
top_x = 10
print(len(dv_tbl2))
print(" TOP %d ASTEROIDS THAT ARE EASIEST TO REACH" % top_x)
for i in range(top_x):
    x = dv_tbl2[i]
    print("%d. %s:  fly-by delta-v=%s randezvous delta-v=%s" % (i+1, x[0], x[3], x[1] ))

In [ ]:
f = open("../data/mpc/asteroids.txt", "w+")
f.write("#name, total delta-v[m/s], time of flight[days], Hohmann burn1 [m/s], Hohmann burn2 [m/s]\n")
for x in dv_tbl2:
    f.write("%s, %4.2f, %4.1f, %4.2f, %4.2f\n" % (x[0], x[1].value, x[2].value, x[3].value, x[4].value))
f.close()